## Twitter Sentiment Analysis

### Importing Dependencies

In [4]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Processing

In [7]:
data = pd.read_csv('data.csv', encoding='ISO-8859-1', header=None)

In [8]:
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
data[0].unique()

array([0, 4])

In [16]:
data.shape

(1600000, 6)

In [29]:
# Adding column names to the data
columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
data.columns = columns

In [30]:
data.sample(5)

,target,ids,date,flag,user,text
24392,0,1557993282,Sun Apr 19 06:52:07 PDT 2009,NO_QUERY,leahstammer,@laurenbeth7293 ah stop direct messaging me it...
1547749,4,2182838693,Mon Jun 15 13:42:10 PDT 2009,NO_QUERY,missclick,http://twitpic.com/7hp58 - Love this....
27759,0,1559352819,Sun Apr 19 10:55:05 PDT 2009,NO_QUERY,LenStorm,@VoiceofCritics ok. gonna have to send it tomo...
497973,0,2186057161,Mon Jun 15 18:17:38 PDT 2009,NO_QUERY,iamjesschiriano,Atheletics carnival's suck! I'm so cold. ryan...
252858,0,1983844278,Sun May 31 13:56:50 PDT 2009,NO_QUERY,C9_H13_NO3_Rush,wishing my fellow skydiver (TS) a speedy recov...


In [31]:
# Dropping unnecessary columns
data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [32]:
# Distribution of target variable
data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

If the taget values weren't equal then by upsampling or by downsampling convert it equale values 

In [33]:
data.replace({'target': {4:1}}, inplace=True)

In [43]:
data.sample(5)

,target,ids,date,flag,user,text
136837,0,1880187409,Fri May 22 00:22:57 PDT 2009,NO_QUERY,papercatss,@simplymallory you be naht online D: sighs i...
1121134,1,1974071332,Sat May 30 12:23:00 PDT 2009,NO_QUERY,krassyo,"@epistemic_murk ?? ??? ????, ?? ????? ??? ?? ?..."
615379,0,2226251693,Thu Jun 18 11:43:17 PDT 2009,NO_QUERY,kinzyy,Its pouring rain and all i want is potbelly an...
848994,1,1564961422,Mon Apr 20 05:29:33 PDT 2009,NO_QUERY,Sir_Almo,@TheRazz huge crowds should be fun
1476497,1,2066102399,Sun Jun 07 09:53:36 PDT 2009,NO_QUERY,FreezDay,@esmeeworld hi the hangover? it's 2009 film?


In [44]:
data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

### Stemming

In [45]:
stemmer = PorterStemmer()

In [48]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [52]:
print("Before Stemming: ", data['text'][0])
print("After Stemming: ", stemming(data['text'][0]))

Before Stemming:  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
After Stemming:  switchfoot http twitpic com zl awww bummer shoulda got david carr third day


In [53]:
data['stemmed_text'] = data['text'].apply(stemming)

In [54]:
data.sample(5)

,target,ids,date,flag,user,text,stemmed_text
813807,1,1548887986,Fri Apr 17 22:17:43 PDT 2009,NO_QUERY,cutieissa,2 ways to earn off Twitter: http://tr.im/hIHv ...,way earn twitter http tr im hihv http gd irtr ...
890199,1,1687971642,Sun May 03 10:02:10 PDT 2009,NO_QUERY,kimroset,is heading off to the Casino! hit me! #fb,head casino hit fb
906046,1,1695279967,Mon May 04 05:43:43 PDT 2009,NO_QUERY,PoppyJH,is at her nan's house eating fish and chips an...,nan hous eat fish chip watch top gear good time
768667,0,2301076874,Tue Jun 23 14:58:14 PDT 2009,NO_QUERY,OMGkayleebee,Harry potter and the prisoner of azkaban. Bore...,harri potter prison azkaban bore im still sick...
1237272,1,1992996652,Mon Jun 01 09:39:49 PDT 2009,NO_QUERY,epitomeofmichie,have yu ever just sat n yur car before getting...,yu ever sat n yur car get amp see thee crazies...


In [55]:
x = data['stemmed_text'].values
y = data['target'].values

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2, stratify=y)

In [57]:
x.shape, x_train.shape, x_test.shape

((1600000,), (1280000,), (320000,))

In [58]:
# Converting text data to numerical data
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

### Train the model

Logistic Regression

In [59]:
model = LogisticRegression(max_iter=1000)

In [60]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [61]:
predict = model.predict(x_test)
accuracy = accuracy_score(y_test, predict)

In [62]:
print("Accuracy: ", accuracy)

Accuracy:  0.77668125
